# You mean really?
*Are we gonna implement the MLR??*

*NOTE*: The implementation is not stated in the book, but from the lectures that I am taking from the GRRAS have taught how to... there are 2 methods (Not with Gredient Descent)
1. With the formula
2. Metrix Method

Here we will implement the 1st method and then in the next book, we will get the 2nd done. <br>
**The most exciting** part here is that, **we are implementing MLR WITH THE 11th formula!!**

## 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Theory
—
### How?
The technique is simple, there is not a big change in the 11th formula... just that is that, **we need to find `coefs` for each features** by applying the same 11th formula to the individual columns.

Then, we will calculate the B0 `intercept` only once (of course) from all the pairs of coefs and means of x.

### Here's more How.
# 
##### COEFS (individual)
## $$ \beta_n = \frac {\sum (x_{ni} - \bar x_n)(y_{i} - \bar y)} {\sum(x_{ni} - \bar x_n)^2} $$

##### INTERCEPT (for all)
## $$ \beta_0 = \bar y - \beta_1 \bar x_1 + \beta_2 \bar x_2 +  ...  +\beta_n \bar x_n $$

# 

Are you ready? Let's go.

In [9]:
df = pd.DataFrame(np.random.randint(0, 100, (10, 3)), columns=["x1", "x2", "x3"])

In [10]:
df["y"] = (2 * df + np.random.randint(0, 100, (10, 1))).sum(1)
df

,x1,x2,x3,y
0,36,93,59,586
1,87,55,32,630
2,33,40,88,580
3,8,30,15,217
4,74,6,47,380
5,95,71,49,559
6,55,16,85,564
7,8,83,7,301
8,96,72,74,745
9,48,48,13,428


Great. Now, we have the ready data. Let's check how can we do it.

# 

In [81]:
class MLR_11th:
    def __init__(self, X: np.ndarray, y: np.ndarray):
        self.x = np.array(X)
        self.y = np.array(y)
        
        if self.x.ndim == 1:
            raise NotImplementedError("""X must not be 1D.
            Hint: Use X[:, np.newaxis] to make it 2D.""")
        elif (len(self.x) != len(y)) or (y.ndim != 1):
            raise NotImplementedError("""Y must have same length
            as the feature does and also it has to be 1D.""")
        else:
            self.coefs = self.calculate_coefs()
            self.intercept = self.calculate_intercept()

    def calculate_coefs(self):
        coefs = []
        self.means = []
        self.y_mean = self.y.mean()
        
        for i in range(self.x.shape[1]):
            xn = self.x[:, i]
            xn_mean = xn.mean()
            self.means.append(xn_mean)
            
            coef = sum((xn - xn_mean) * (self.y - self.y_mean)) / sum((xn - xn_mean) ** 2)
            coefs.append(coef)
        return coefs    
    
    def calculate_intercept(self):
        coef_into_mean = 0
        for coef, mean in zip(self.coefs, self.means):
            coef_into_mean += coef * mean
        return self.y_mean - coef_into_mean
    
    def predict(self, X):
        X = X.values
        predict = []
        
        for record in X:
            coefn_into_xn = 0
            for coef, xn in zip(self.coefs, record):
                coefn_into_xn += coef * xn
            predict.append(self.intercept + coefn_into_xn )
        return predict

In [73]:
model = MLR_11th(df.iloc[:, :-1], df.y)

In [74]:
model.coefs

[3.414827447833066, 1.6789025239804938, 3.870411769848505]

In [75]:
model.intercept

46.78141607852217

# 

## Trying with `sklearn`

In [40]:
from sklearn.linear_model import LinearRegression

In [61]:
model = LinearRegression()
model.fit(df.iloc[:, :-1], df.y)

LinearRegression()

In [62]:
model.coef_

array([2.39399049, 2.25548065, 3.26229756])

In [63]:
model.intercept_

100.79105261984853

**Nah!!** The result is not the same... let's test on the same data and then check the r2 score.

# 

# Checking for $R^2$ 

Our model

In [86]:
model = MLR_11th(df.iloc[:, :-1], df.y)

In [87]:
pred_1 = model.predict(df.iloc[:, :-1])
pred_1

[554.2074333517603,
 560.0642194940782,
 567.2230585629015,
 182.5232879283291,
 491.4614155449318,
 680.0422795478553,
 590.4443665301517,
 240.5418275405072,
 781.8963037658815,
 341.59580773360364]

##### 

Their model

In [88]:
model = LinearRegression()
model.fit(df.iloc[:, :-1], df.y)

LinearRegression()

In [90]:
pred_2 = model.predict(df.iloc[:, :-1])
pred_2

array([589.20996672, 537.51318294, 557.09414984, 236.54185944,
       444.80721798, 648.21185575, 545.84351232, 329.98395353,
       734.4187658 , 366.37553568])

# 

Checking...

In [91]:
from sklearn.metrics import r2_score

In [92]:
r2_score(df.y, pred_1)

0.800009799541376

In [93]:
r2_score(df.y, pred_2)

0.8875717796036926

# 

sklearn's does better... why?